# Test Construction Across Multiple Domains and Difficulties

In [4]:
from pydantic import BaseModel, ValidationError
from typing import List
import pandas as pd

class MultipleChoiceQuestion(BaseModel):
    item_id: str
    stem: str  # The actual question text
    options: List[str]  # The list of answer choices
    key_index: int  # Index of the correct answer in the options list
    difficulty_level: str = None  # Optional difficulty level of the question
    skill_domain: str = None  # Optional specific skill or knowledge domain

# Read the CSV file using Pandas
df = pd.read_csv('./train.csv')

# We don't have difficulty_level and skill_domain provided in the CSV, so we'll skip these.
# If available, they could be added similarly.

# Transform the data into MultipleChoiceQuestion instances
mcq_list = []
for _, row in df.iterrows():
    options = [row['A'], row['B'], row['C'], row['D'], row['E']]
    answer_letter = row['answer'].upper()  # Ensure it's uppercase
    key_index = ord(answer_letter) - ord('A')  # Convert letter to index
    try:
        mcq_item = MultipleChoiceQuestion(
            item_id=str(row['id']),
            stem=row['prompt'],
            options=options,
            key_index=key_index,
        )
        mcq_list.append(mcq_item)
    except ValidationError as e:
        print(f"Error processing row with id {row['id']}: {e}")

# mcq_list now contains a list of MultipleChoiceQuestion instances

In [6]:
mcq_list[19]

MultipleChoiceQuestion(item_id='19', stem='What is reciprocal length or inverse length?', options=['Reciprocal length or inverse length is a quantity or measurement used in physics and chemistry. It is the reciprocal of time, and common units used for this measurement include the reciprocal second or inverse second (symbol: s−1), the reciprocal minute or inverse minute (symbol: min−1).', 'Reciprocal length or inverse length is a quantity or measurement used in geography and geology. It is the reciprocal of area, and common units used for this measurement include the reciprocal square metre or inverse square metre (symbol: m−2), the reciprocal square kilometre or inverse square kilometre (symbol: km−2).', 'Reciprocal length or inverse length is a quantity or measurement used in biology and medicine. It is the reciprocal of mass, and common units used for this measurement include the reciprocal gram or inverse gram (symbol: g−1), the reciprocal kilogram or inverse kilogram (symbol: kg−1)

What makes a multiple choice question easy for a human? For a language model? Medium? Hard?

In [ ]:
data = {
    "question": {
        "id": "unique_identifier",
        "text": "What is the capital of France?",
        "domain": "Geography",
        "subdomain": "European Capitals",
        "difficulty_level": "Easy",
        "tags": ["Capital Cities", "Europe", "World Geography"]
    },
    "options": [
        {"id": "A", "text": "Paris", "is_correct": True},
        {"id": "B", "text": "Berlin", "is_correct": False},
        {"id": "C", "text": "Madrid", "is_correct": False},
        {"id": "D", "text": "Rome", "is_correct": False}
    ],
    "explanation": {
        "text": "Paris is the capital of France. Berlin is the capital of Germany, Madrid is the capital of Spain, and Rome is the capital of Italy.",
        "references": [
            {"source": "World Atlas", "url": "https://www.worldatlas.com/europe/france"}
            # Other references can be included as necessary
        ]
    },
    "metadata": {
        "author": "QuizMaster",
        "creation_date": "2023-01-01",
        "last_update": "2023-01-05",
        "usage": {"times_used": 100, "average_score": 0.85}
    }
}


In [ ]:
from typing import List, Optional
from pydantic import BaseModel, HttpUrl, validator

class Option(BaseModel):
    id: str
    text: str
    is_correct: bool

class ExplanationReference(BaseModel):
    source: str
    url: HttpUrl

class Explanation(BaseModel):
    text: str
    references: List[ExplanationReference]

class UsageMetadata(BaseModel):
    times_used: int
    average_score: float

class QuestionMetadata(BaseModel):
    author: str
    creation_date: str  # Could be a datetime field if you require a datetime object
    last_update: str
    usage: UsageMetadata

class MultipleChoiceQuestion(BaseModel):
    id: str
    text: str
    domain: str
    subdomain: Optional[str]
    difficulty_level: str
    tags: List[str]
    options: List[Option]
    explanation: Explanation
    metadata: QuestionMetadata

    @validator('options')
    def validate_options(cls, options):
        correct_options = [option for option in options if option.is_correct]
        if len(correct_options) != 1:
            raise ValueError('There must be exactly one correct option')
        return options

In [9]:
from typing import List
from pydantic import BaseModel, HttpUrl, validator
from datetime import date

# Base models
class BaseOption(BaseModel):
    id: str
    text: str

class CorrectOption(BaseOption):
    is_correct: bool = True

class IncorrectOption(BaseOption):
    is_correct: bool = False

class Reference(BaseModel):
    source: str
    url: HttpUrl

class Explanation(BaseModel):
    text: str
    references: List[Reference]

class UsageMetadata(BaseModel):
    times_used: int
    average_score: float

class Metadata(BaseModel):
    author: str
    creation_date: date
    last_update: date
    usage: UsageMetadata

# Question model with inheritance
class MultipleChoiceQuestion(BaseModel):
    id: str
    text: str
    domain: str
    subdomain: str = None
    difficulty_level: str
    tags: List[str]
    correct_option: CorrectOption
    incorrect_options: List[IncorrectOption]
    explanation: Explanation
    metadata: Metadata

    @property
    def options(self):
        return [self.correct_option] + self.incorrect_options

    @validator('incorrect_options', pre=True)
    def validate_correct_option(cls, incorrect_options):
        if not any(o.is_correct for o in incorrect_options):
            raise ValueError('There must be exactly one correct option')
        return incorrect_options
    
    class Config:
        validate_assignment = True

/tmp/ipykernel_21712/1511559684.py:51: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  @validator('incorrect_options', pre=True)


In [ ]:
# Example usage:

question = MultipleChoiceQuestion(
    id="q1",
    text="What is the capital of France?",
    domain="Geography",
    difficulty_level="Easy",
    tags=["Capital Cities", "Europe", "World Geography"],
    correct_option=CorrectOption(
        id="A",
        text="Paris",
    ),
    incorrect_options=[
        IncorrectOption(id="B", text="Berlin"),
        IncorrectOption(id="C", text="Madrid"),
        IncorrectOption(id="D", text="Rome"),
    ],
    explanation=Explanation(
        text="Paris is the capital of France. Berlin is the capital of Germany, Madrid is the capital of Spain, and Rome is the capital of Italy.",
        references=[
            Reference(
                source="World Atlas",
                url="https://www.worldatlas.com/europe/france"
            ),
            # Other references can be included as necessary
        ]
    ),
    metadata=Metadata(
        author="QuizMaster",
        creation_date=date(2023, 1, 1),
        last_update=date(2023, 1, 5),
        usage=UsageMetadata(
            times_used=100,
            average_score=0.85
        )
    )
)

# Printing the question to verify the structure
print(question.json(indent=2))